# Kaggle Score

![alt text](kaggle_result.png "Title")

## TOP 3 Submission:
### 0.15122 (Architecture : 2000 Neurons, 200 Epochs, learning rate : 0.001 ) : 
This model model converged best with the above setup. As learning rate is sufficient for model to learn also sufficient neurons to fire.
### 0.15244 (Architecture : 2000 Neurons, 250 Epochs, learning rate : 0.001 ) :
This model maybe gets little more converged than the above one as it has same architecture but it is running for 250 epochs 
### 0.19819 (Architecture : 1500 Neurons, 150 Epochs, learning rate : 0.001 ) :
This model has less neurons and also less epochs so less neurons are firing and also the model is not fully converged.

In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.nn.functional as F
from sklearn import preprocessing 
import torch.utils.data as utils
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
import torchvision

# Loading Files and droping ID column from train

In [2]:
train_dataset = pd.read_csv('./dataset/kaggle/train.csv',sep=',')

test_dataset = pd.read_csv('./dataset/kaggle/test.csv',sep=',')
test_dataset = test_dataset.drop('Id',axis=1)
label = train_dataset.SalePrice
train_dataset = train_dataset.drop(['SalePrice','Id'],axis=1)

In [3]:
train_dataset.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


# Replace Categorical value with NA

In [4]:
def changeNanCategorical(dataset):
    categorical = ['Alley','FireplaceQu','Fence','MiscFeature','PoolQC','GarageQual','GarageFinish','GarageType','GarageCond','BsmtQual','BsmtCond','BsmtFinType1','BsmtFinType2','BsmtExposure']
    for col in categorical:
        dataset[col].fillna('NA', inplace=True)
    return dataset


# Replace Numerical value with the most frequent value.

In [5]:
def changeNanNumerical(dataset,y):
    for i in range(y):
        #print(test_dataset.isna().sum().sort_values()[-18:-1].to_frame().index[i])
        col = dataset.isna().sum().sort_values()[-(y+2):-1].to_frame().index[i]
        dataset[col].fillna(dataset[col].value_counts().to_frame().index[0],inplace=True)
    return dataset

# TestData replacing values

In [6]:
test_dataset = changeNanCategorical(test_dataset)
test_dataset = changeNanNumerical(test_dataset,78)
test_dataset.LotFrontage.fillna(test_dataset.LotFrontage.value_counts().to_frame().index[0],inplace=True)

# TrainData replacing values

In [7]:
train_dataset = changeNanCategorical(train_dataset)
train_dataset = changeNanNumerical(train_dataset,78)
train_dataset.LotFrontage.fillna(train_dataset.LotFrontage.value_counts().to_frame().index[0],inplace=True)

# Get categorical and numerical column

In [8]:
def getCategoricalColumn(dataset):
    column = []
    for col in dataset.columns:
        if isinstance(dataset[col].unique()[0], str):
            column.append(col)
    return column
categoricalColum = getCategoricalColumn(train_dataset)
num_cols = train_dataset._get_numeric_data().columns

# Create Dummy dataframe 

In [9]:
X_train_num = pd.DataFrame()
X_train_Cat = pd.DataFrame()
X_test_num = pd.DataFrame()
X_test_Cat = pd.DataFrame()

# Add Numerical Column

In [10]:
for i in num_cols:
    X_train_num[i] = train_dataset[i]
    X_test_num[i] = test_dataset[i]

# Normalise the Numerical data

In [11]:
X_train_num = (X_train_num-X_train_num.mean())/X_train_num.std()
X_test_num = (X_test_num-X_test_num.mean())/X_test_num.std()

In [12]:
# Add Categorical Column

In [13]:
for i in categoricalColum:
    X_train_Cat[i] = train_dataset[i]
    X_test_Cat[i] = test_dataset[i]

# One Hot Encoding on Test and Train Data

In [14]:
def changeStrColInOneHotEncoding(dataset):
    for col in dataset.columns:
        if isinstance(dataset[col].unique()[0], str):
            # Get one hot encoding of columns B
            one_hot = pd.get_dummies(dataset[col],prefix=str(col+"_"))
            # Drop column B as it is now encoded
            dataset = dataset.drop(col,axis = 1)
            # Join the encoded df
            dataset = dataset.join(one_hot)
    return dataset


X_train_Cat = changeStrColInOneHotEncoding(X_train_Cat)
X_test_Cat = changeStrColInOneHotEncoding(X_test_Cat)

# Concat Categorical and numerical column

In [15]:
X_train = pd.concat([X_train_Cat, X_train_num], axis=1)
X_test = pd.concat([X_test_Cat, X_test_num], axis=1)

# Add column in Test data which are not present from Train

In [16]:
for i in  X_train.columns:
    if i not in X_test.columns:
        ind = X_train.columns.get_loc(i)
        X_test.insert(ind,i,0)

In [17]:
X_train.isna().sum().any()

False

In [18]:
X_train = X_train.values
X_test = X_test.values

In [19]:
X_train.shape

(1460, 302)

# Load the data in torch tensor

In [20]:
torch_tensor = torch.tensor(X_train)
torch_tensor = torch_tensor.type(torch.FloatTensor)
label = label.values.astype(np.float)
label_tensor = torch.tensor(label)
label_tensor =  label_tensor.type(torch.FloatTensor)

tensor_x,tensor_y = torch_tensor,label_tensor
my_dataset = utils.TensorDataset(tensor_x,tensor_y) # create your datset
train_loader = utils.DataLoader(my_dataset,batch_size=1) # create your dataloader

# Make a FC Model

In [21]:
class Net(nn.Module):
    def __init__(self,size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(size, 2500)  
        #self.fc2 = nn.Linear(1024, 256)
        #self.fc3 = nn.Linear(256, 128)
        #self.fc4 = nn.Linear(128, 16)
        self.fc5 = nn.Linear(2500,1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        #x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x


model = Net(tensor_x.shape[1])
model = model.cuda()
print(model)

Net(
  (fc1): Linear(in_features=302, out_features=2500, bias=True)
  (fc5): Linear(in_features=2500, out_features=1, bias=True)
)


# Initialize criterion and optimizer

In [22]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model

In [23]:
for epoch in range(250):
    ave_loss=0.0
    
    for i,data in enumerate(train_loader,0):
        inputs,target =data
        inputs = inputs.float().cuda()
        target = target.float().cuda()
        optimizer.zero_grad()
        output=model(inputs)
        output = output.squeeze(1)
        loss=criterion(output,target)
        loss.backward()
        optimizer.step()
        ave_loss+=loss.item()
    print("Iter ",epoch," loss : ",ave_loss/train_dataset.shape[0])

Iter  0  loss :  19936008237.978073
Iter  1  loss :  3096127447.4531803
Iter  2  loss :  1974120493.7309115
Iter  3  loss :  1589802617.1549587
Iter  4  loss :  1389894772.5745742
Iter  5  loss :  1259415765.2417643
Iter  6  loss :  1168705438.8682396
Iter  7  loss :  1103164201.9076674
Iter  8  loss :  1054047235.1013278
Iter  9  loss :  1015957556.9525133
Iter  10  loss :  985473204.5102208
Iter  11  loss :  960395090.0405735
Iter  12  loss :  939251864.8032942
Iter  13  loss :  921046592.986016
Iter  14  loss :  905101207.5306957
Iter  15  loss :  890936516.7469901
Iter  16  loss :  878222944.9893478
Iter  17  loss :  866708703.2952877
Iter  18  loss :  856202500.1868601
Iter  19  loss :  846550502.2017279
Iter  20  loss :  837634214.839999
Iter  21  loss :  829357069.1709275
Iter  22  loss :  821640221.2131428
Iter  23  loss :  814415028.0842257
Iter  24  loss :  807627438.1822406
Iter  25  loss :  801227199.807112
Iter  26  loss :  795173220.6412601
Iter  27  loss :  789426264.985

Iter  225  loss :  496901679.3773233
Iter  226  loss :  496140831.78981483
Iter  227  loss :  495379276.22507375
Iter  228  loss :  494619378.4544646
Iter  229  loss :  493862930.29593015
Iter  230  loss :  493097044.4413213
Iter  231  loss :  492340981.70412195
Iter  232  loss :  491582621.4101811
Iter  233  loss :  490828641.2774133
Iter  234  loss :  490067220.0746915
Iter  235  loss :  489308762.0377131
Iter  236  loss :  488548510.0053115
Iter  237  loss :  487796427.9520482
Iter  238  loss :  487028741.34611434
Iter  239  loss :  486269490.3603611
Iter  240  loss :  485508052.161582
Iter  241  loss :  484742547.3467497
Iter  242  loss :  483970007.0017275
Iter  243  loss :  483212386.72367364
Iter  244  loss :  482440151.53529155
Iter  245  loss :  481671293.1399789
Iter  246  loss :  480903486.2507883
Iter  247  loss :  480150855.05673814
Iter  248  loss :  479361014.43455374
Iter  249  loss :  478597854.83006394


In [24]:
torch_tensor = torch.Tensor(X_test)
x_test = torch_tensor.type(torch.FloatTensor)
print(x_test.shape)

torch.Size([1459, 302])


# Predict from model

In [25]:
predict_test = model(x_test.cuda())

In [26]:
predict_test

tensor([[108431.2500],
        [149578.7656],
        [182792.8906],
        ...,
        [162402.4531],
        [109577.7969],
        [216829.1562]], device='cuda:0', grad_fn=<AddmmBackward>)

In [27]:
predict_test = predict_test.cpu()
predict_test = predict_test.detach().numpy() 

In [30]:
predict_test.max()

636245.3

# Save in CSV

In [29]:
test_df = pd.DataFrame(columns=['Id', 'SalePrice'])

test_dataset1 = pd.read_csv('./dataset/kaggle/test.csv',sep=',')
test_df['Id'] = test_dataset1.Id

test_df['SalePrice'] = predict_test

test_df.to_csv("submission.csv",index=False)